<a href="https://colab.research.google.com/github/sn0005/Python-Programming/blob/Practice%2FProjects/webscrappingwithbeautifulsoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing required libraries
import pandas as pd
import numpy as np
from requests.api import request
from requests import get
from bs4 import BeautifulSoup
from time import time, sleep
from random import randint
from IPython.core.display import clear_output
from warnings import warn
warn("Warning Simulation")

header = {'Accept-Language': "en-US, en;q=0.5"} # to avoid some of the movie names translated into the main language of the country from where the code is ran.

#Declaring variables to contain data in.
Name = []
Year = []
iMDB = []
Metascore = []
Rating = []

#creating variables for number of pages we want to look and which all years movie data we want to collect. 
pages = [str(i) for i in range(1,3)]
years_url = [str(i) for i in range(2016,2021)]

#preparing to monitor the loop.
start_time = time()

requests = 0

#our loop.
for year in years_url:
  for page in pages:
    
    #making get request
    response = get('https://www.imdb.com/search/title/?release_date='+ year + '&sort=num_votes,desc&page=' + page, headers = header)
    
    #controlling craw-rate
    sleep(randint(8,15))
    
    #monitor requests.
    requests += 1
    current_time = time()
    elapsed_time = current_time - start_time
    print('Request {}, Frequency {}'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    #to throw warning for non 200 status codes.
    if response.status_code != 200:
      warn('Request {}, Frequency {}'.format(requests, response.status_code))

    #controlling the loop if requests exceed expected 
    if requests > 20:
      warn('Number of requests was greater than expected.')
      break
    
    #parse content of request with BeautifulSoup.
    html_soup = BeautifulSoup(response.text, 'html.parser')

    #select all movies from single page
    movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced') 
    
    #for every movie from above page.
    for container in movie_containers:
      if container.find('div', class_ = 'inline-block ratings-metascore') is not None:
        #scrape Name
        Name.append(container.h3.a.text)
        #scrape Year
        Year.append(container.find('span', class_='lister-item-year text-muted unbold').text)
        #scrape imdb
        iMDB.append(float(container.strong.text))
        #scrape metascore
        Metascore.append(int(container.find('span', class_='metascore').text))
        #scrape rating
        Rating.append(int(container.find('span', attrs = {'name': 'nv'})['data-value']))

Request 10, Frequency 0.0853485877958941


In [ ]:
#creating dataframe
import pandas as pd
movie_ratings = pd.DataFrame({'movie': Name, 'year':Year, 'imdb': iMDB, 'metascore': Metascore, 'vote': Rating})
movie_ratings.head()

,movie,year,imdb,metascore,vote
0,Deadpool,(2016),8.0,65,966967
1,Captain America: Civil War,(2016),7.8,75,725699
2,Batman v Superman: Dawn of Justice,(2016),6.4,44,671501
3,Doctor Strange,(2016),7.5,72,663251
4,Suicide Squad,(2016),5.9,40,659477


In [ ]:
#converting test_df['year'] dtype to int
movie_ratings['year'] = [int(value[-5:-1]) for value in test_df['year'].values]

#another way of converting dtype
#www.dataquest.io/blog/web-scraping-beautifulsoup
#movie_ratings.loc[:, 'year'] = movie_ratings['year'].str[-5:-1].astype(int)

In [ ]:
movie_ratings.head(10)

,movie,year,imdb,metascore,vote
0,Deadpool,2016,8.0,65,966967
1,Captain America: Civil War,2016,7.8,75,725699
2,Batman v Superman: Dawn of Justice,2016,6.4,44,671501
3,Doctor Strange,2016,7.5,72,663251
4,Suicide Squad,2016,5.9,40,659477
5,Arrival,2016,7.9,81,652281
6,Rogue One: A Star Wars Story,2016,7.8,65,589226
7,La La Land,2016,8.0,94,548000
8,Hacksaw Ridge,2016,8.1,71,479535
9,Zootopia,2016,8.0,78,466284


In [ ]:
#writing to csv file
movie_ratings.to_csv("movie_ratings.csv")